
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# Lab: Preparing Data for RAG

The objective of this lab is to demonstrate the process of ingesting and processing documents for a Retrieval-Augmented Generation (RAG) application. This involves extracting text from PDF documents, computing embeddings using a foundation model, and storing the embeddings in a Delta table.


**Lab Outline:**

In this lab, you will need to complete the following tasks:

* **Task 1 :** Read the PDF files and load them into a DataFrame.

* **Task 2 :** Extract the text content from the PDFs and split it into manageable chunks.

* **Task 3 :** Compute embeddings for each text chunk using a foundation model endpoint.

* **Task 4 :** Create a Delta table to store the computed embeddings.

**📝 Your task:** Complete the **`<FILL_IN>`** sections in the code blocks and follow the other steps as instructed.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the lab, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%pip install -qq -U llama-index pydantic PyPDF2
dbutils.library.restartPython()

In [0]:
%run ../Includes/Classroom-Setup-02

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

## Task 1: Read the PDF files and load them into a DataFrame.

To start, you need to load the PDF files into a DataFrame.

**Steps:**

1. Use Spark to load the binary PDFs into a DataFrame.

2. Ensure that each PDF file is represented as a separate record in the DataFrame.

In [0]:
# run this cell to import the required libraries
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import Document
from llama_index.core.utils import set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator
from pyspark.sql.functions import col, udf, length, pandas_udf, explode
import os
import pandas as pd 
import io
from PyPDF2 import PdfReader

In [0]:
# use Spark to load the PDF files into a DataFrame
# reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)
articles_path = f"{DA.paths.datasets.arxiv}/arxiv-articles/"
table_name = f"{DA.catalog_name}.{DA.schema_name}.lab_pdf_raw_text"

# read pdf files
df = (
        spark.read.format("binaryfile")
        .option("recursiveFileLookup", "true")
        .load(articles_path)
        )

# save list of the files to table
df.write.mode("overwrite").saveAsTable(table_name)

display(df)

## Task 2: Extract the text content from the PDFs and split it into manageable chunks

Next, extract the text content from the PDFs and split it into manageable chunks.

**Steps:**

1. Define a function to split the text content into chunks.

    * Split the text content into manageable chunks.

    * Ensure each chunk contains a reasonable amount of text for processing.

2. Apply the function to the DataFrame to create a new DataFrame with the text chunks.


In [0]:
# Define a function to split the text content into chunks
@pandas_udf("array<string>")
def read_as_chunk(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Set llama2 as tokenizer
    set_global_tokenizer(
      AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    # Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    
    def extract_and_split(b):
        txt = parse_bytes_pypdf(b)
        if txt is None:
            return []
        nodes = splitter.get_nodes_from_documents([Document(text=txt)])
        return [n.text for n in nodes]

    for x in batch_iter:
        yield x.apply(extract_and_split)

df_chunks = (df
              .withColumn("content", explode(read_as_chunk("content")))
              .selectExpr("path as pdf_name", "content")
            )
display(df_chunks)

## Task 3: Compute embeddings for each text chunk using a foundation model endpoint
Now, compute embeddings for each text chunk using a foundation model endpoint.

**Steps:**

1. Define a function to compute embeddings for text chunks.
    + Use a foundation model endpoint to compute embeddings for each text chunk.
    + Ensure that the embeddings are computed efficiently, considering the limitations of the model.  

2. Apply the function to the DataFrame containing the text chunks to compute embeddings for each chunk.


In [0]:
# define a function to compute embeddings for text chunks
@pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    
    def get_embeddings(batch):
        # calculate embeddings using the deployment client's predict function 
        # NOTE: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-gte-large-en", inputs={"input": batch})
        return [e["embedding"] for e in response.data]

    # splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)
    
df_chunk_emd = (df_chunks
                .withColumn("embedding", get_embedding("content"))
                .selectExpr("pdf_name", "content", "embedding")
                )
display(df_chunk_emd)

## Task 4: Create a Delta table to store the computed embeddings

Finally, create a Delta table to store the computed embeddings.

Steps:

  1. Define the schema for the Delta table.

  2. Save the DataFrame containing the computed embeddings as a Delta table.


**Note:** Ensure that the Delta table is properly structured to facilitate efficient querying and retrieval of the embeddings.

**📌 Instructions:** 

- Please execute the following SQL code block to create the Delta table. This table will store the computed embeddings along with other relevant information. 

**Important:** Storing the computed embeddings in a structured format like a Delta table ensures efficient querying and retrieval of the embeddings when needed for various downstream tasks such as retrieval-augmented generation. Additionally, setting the `delta.enableChangeDataFeed` property to true enables Change Data Feed (CDC), which is required for VectorSearch to efficiently process changes in the Delta table.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS lab_pdf_text_embeddings (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  pdf_name STRING,
  content STRING,
  embedding ARRAY <FLOAT>
  -- NOTE: the table has to be CDC because VectorSearch is using DLT that is requiring CDC state
  ) TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
# define the schema for the Delta table
embedding_table_name = f"{DA.catalog_name}.{DA.schema_name}.lab_pdf_text_embeddings"
# save the DataFrame as a Delta table
df_chunk_emd.write.mode("append").saveAsTable(embedding_table_name)


## Clean up Classroom

**🚨 Warning:** Please refrain from deleting tables created in this lab, as they are required for upcoming labs. To clean up the classroom assets, execute the classroom clean-up script provided in the final lab.


## Conclusion

In this lab, you learned how to prepare data for Retrieval-Augmented Generation (RAG) applications. By extracting text from PDF documents, computing embeddings, and storing them in a Delta table, you can enhance the capabilities of language models to generate more accurate and relevant responses.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>